In [ ]:
from __future__ import division
from __future__ import print_function

import time
import argparse
import numpy as np

import torch
import torch.nn.functional as F
import torch.optim as optim
import random
from models import GCN

import matplotlib.pyplot as plt

from sklearn import metrics
from utils import get_plot


In [ ]:
from data_process import generate_data, load_data
from train_func import test, train, Block_matrix_train, Lhop_Block_matrix_train, Communicate_train, ADMM_communication_train, Block_matrix_train_batch

In [ ]:
def get_K_hop_neighbors(adj_matrix, index, K):
    adj_matrix = adj_matrix + torch.eye(adj_matrix.shape[0],adj_matrix.shape[1])  #make sure the diagonal part >= 1
    hop_neightbor_index=index
    for i in range(K):
        hop_neightbor_index=torch.unique(torch.nonzero(adj[hop_neightbor_index])[:,1]) #include 1-k hop neightbors
    return hop_neightbor_index

In [ ]:
def get_gradient_two_layer(features, adj_2_hop, adj_1_hop, number_of_datapoints):
    #adj_2_hop = normalize(adj_2_hop)
    #adj_1_hop = normalize(adj_1_hop)
    return torch.norm(features.T.mm(adj_2_hop.T).mm(adj_1_hop.T).mm(adj_1_hop).mm(adj_2_hop).mm(features)) / number_of_datapoints

In [ ]:
import scipy.sparse as sp
def normalize(mx):
    """Row-normalize sparse matrix"""
    
    mx = mx + torch.eye(mx.shape[0],mx.shape[1])
    
    rowsum = np.array(mx.sum(1))
    r_inv = np.power(rowsum, -1).flatten()
    r_inv[np.isinf(r_inv)] = 0.
    r_mat_inv = sp.diags(r_inv)
    mx = r_mat_inv.dot(mx)
    return torch.tensor(mx)

In [ ]:

def get_graph(dataset_name, iid_percent, K_over_class_num):
    
        if dataset_name=='simulate':
            

            number_of_nodes=200
            class_num=3
            link_inclass_prob=10/number_of_nodes  #when calculation , remove the link in itself
            #EGCN good when network is dense 20/number_of_nodes  #fails when network is sparse. 20/number_of_nodes/5
            link_outclass_prob=link_inclass_prob/20
            features, adj, labels, idx_train, idx_val, idx_test =generate_data(number_of_nodes,  class_num, link_inclass_prob, link_outclass_prob)               
        else:
            #'cora', 'citeseer', 'pubmed' #other dataset twitter, 
            

            features, adj, labels, idx_train, idx_val, idx_test = load_data(dataset_name)
            class_num = labels.max().item() + 1


        #client num
        K = K_over_class_num * class_num
        
        
        split_data_indexes=[]
        
        nclass=labels.max().item() + 1
        split_data_indexes = []
        non_iid_percent = 1 - float(iid_percent)
        iid_indexes = [] #random assign
        shuffle_labels = [] #make train data points split into different devices
        for i in range(K):
            current = torch.nonzero(labels == i).reshape(-1)
            current = current[np.random.permutation(len(current))] #shuffle
            shuffle_labels.append(current)
                
        average_device_of_class = K // nclass
        if K % nclass != 0: #for non-iid
            average_device_of_class += 1
        for i in range(K):  
            label_i= i // average_device_of_class    
            labels_class = shuffle_labels[label_i]

            average_num= int(len(labels_class)//average_device_of_class * non_iid_percent)
            split_data_indexes.append(np.array(labels_class[average_num * (i % average_device_of_class):average_num * (i % average_device_of_class + 1)]))
        
        L = []
        for i in split_data_indexes:
            L += list(i)
        L.sort()
        iid_indexes = np.setdiff1d(range(len(labels)), L)
        
        for i in range(K):  #for iid
            label_i= i // average_device_of_class
            labels_class = shuffle_labels[label_i]

            average_num= int(len(labels_class)//average_device_of_class * (1 - non_iid_percent))
            split_data_indexes[i] = list(split_data_indexes[i]) + list(iid_indexes[:average_num])
                    
            iid_indexes = iid_indexes[average_num:]
        
        
        #get train indexes in each device, only part of nodes in each device have labels in the train process
        split_train_ids = []
        for i in range(K):
            split_data_indexes[i].sort()
            inter = np.intersect1d(split_data_indexes[i], idx_train)
            split_train_ids.append(np.searchsorted(split_data_indexes[i], inter))   #local id in block matrix

        one_hot_labels = F.one_hot(labels).float()
        
        return features, adj, labels, one_hot_labels, split_data_indexes, K


# Communication Cost

In [ ]:

import numpy as np
np.random.seed(42)
torch.manual_seed(42)
dataset_name='simulate'
args_normalize = True


for percent in range(0, 11, 1):
    length_current_index = []
    length_neighbor1 = []
    length_neighbor2 = []
    
    for i in range(10):
        iid_percent = percent / 10
        K_over_class_num = 1
        #node_num = 100
        Class_num = 3
        
        features, adj, labels, one_hot_labels, split_data_indexes, K = get_graph(dataset_name, iid_percent, K_over_class_num)


        
        L = []
        for i in range(Class_num-1):
            L+= [i] * int(node_num / Class_num)
        L += [Class_num-1] * (node_num - len(L))
        
        Theta = F.one_hot(torch.tensor(L)).float()
        #L = [1 / Class_num] * Class_num
        #Theta = torch.tensor([L for i in range(node_num)]).float()

        #Lambda = torch.eye(Class_num)
        A = torch.eye(Class_num) * link_inclass_prob

        B = link_outclass_prob * (torch.ones((Class_num, Class_num)) - torch.eye(Class_num))

        Lambda = A+B


        adj = Theta.mm(Lambda).mm(Theta.T)
        features = torch.eye(len(adj))
        one_hot_labels = Theta 
        adj = normalize(adj)
        get_gradient_two_layer(features, adj_2_hop, adj_1_hop, number_of_datapoints)
        
        non_iid_num = int(node_num / Class_num * p)

        Local_index = list(range(int(node_num / Class_num * p))) + list(np.random.randint(int(node_num / Class_num) + 1,node_num, int(node_num / Class_num * (1-p))))
        print(Local_index)


In [ ]:
import numpy as np
np.random.seed(42)
torch.manual_seed(42)
dataset_name='simulate'
args_normalize = True

List_length_current_index = []
List_length_neighbor1 = []
List_length_neighbor2 = []


for percent in range(0, 11, 1):
    length_current_index = []
    length_neighbor1 = []
    length_neighbor2 = []
    
    for i in range(10):
        iid_percent = percent / 10
        K_over_class_num = 1
        features, adj, labels, one_hot_labels, split_data_indexes, K = get_graph(dataset_name, iid_percent, K_over_class_num)
        if dataset_name=='simulate':
                feature = one_hot_labels
        if args_normalize:
            adj = normalize(adj)

        for i in range(len(split_data_indexes)):
                current_index = split_data_indexes[i]

                neighbor1 = get_K_hop_neighbors(adj, current_index, 1)

                neighbor2 = get_K_hop_neighbors(adj, current_index, 2)

                #print(len(current_index), len(neighbor1), len(neighbor2))
                length_current_index.append(len(current_index))
                length_neighbor1.append(len(neighbor1))
                length_neighbor2.append(len(neighbor2))
        print(np.array(length_current_index).sum() / len(length_current_index))
        print(np.array(length_neighbor1).sum() / len(length_neighbor1))
        print(np.array(length_neighbor2).sum() / len(length_neighbor2))
    
    List_length_current_index.append(np.array(length_current_index).sum() / len(length_current_index))
    List_length_neighbor1.append(np.array(length_neighbor1).sum() / len(length_neighbor1))
    List_length_neighbor2.append(np.array(length_neighbor2).sum() / len(length_neighbor2))



In [ ]:
plt.style.use('seaborn-whitegrid')
X = np.array(range(0, 11, 1)) / 10
plt.plot(X, [0]*len(X), 's-', label = 'FedGCN(0-hop)', markersize=8)
plt.plot(X, L, '+-', label = 'FedGCN(1-hop)', markersize=8)
plt.plot(X, np.array(List_length_neighbor1) - np.array(List_length_current_index), '*-', label = 'FedGCN(2-hop)', markersize=8)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
plt.xlabel('IID Degree', fontsize=25)
plt.ylabel('Communication Cost', fontsize=25)
plt.legend(fontsize=20, frameon=True, loc = 'best')

from matplotlib import rcParams
rcParams.update({'figure.autolayout': True})
plt.savefig("simulate_communication_cost_changeIID.png", format='png')

In [ ]:
plt.style.use('seaborn-whitegrid')
X = np.array(range(0, 11, 1)) / 10
plt.plot(X, List_length_current_index, 's-', label = '0-hop', markersize=8)
plt.plot(X, List_length_neighbor1, '+-', label = '1-hop', markersize=8)
plt.plot(X, List_length_neighbor2, '*-', label = '2-hop', markersize=8)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
plt.xlabel('IID Degree', fontsize=25)
plt.ylabel('Number of Nodes', fontsize=25)
plt.legend(fontsize=20, frameon=True, loc = 'best')

from matplotlib import rcParams
rcParams.update({'figure.autolayout': True})
plt.savefig("simulate_node_number_changeIID.png", format='png')

# Convergence Rate

In [ ]:
np.random.seed(42)
torch.manual_seed(42)
dataset_name='simulate'
args_normalize = True
Y_Norm_Local_0hop_Global = []
Y_Norm_Local_1hop_Global = []
Y_Norm_Local_2hop_Global = []

for percent in range(0, 11, 1):
    
    iid_percent = percent / 10
    K_over_class_num = 1
    features, adj, labels, one_hot_labels, split_data_indexes, K = get_graph(dataset_name, iid_percent, K_over_class_num)
    #if dataset_name=='simulate':
    #        feature = one_hot_labels
    if args_normalize:
        adj = normalize(adj)
    Norm_Local_0hop_Global = []
    Norm_Local_1hop_Global = []
    Norm_Local_2hop_Global = []
    
    

    for i in range(len(split_data_indexes)):
            current_index = split_data_indexes[i]
            Local_0hop = get_gradient_two_layer(features[current_index], adj[current_index][:,current_index], adj[current_index][:,current_index], len(current_index))

            neighbor1 = get_K_hop_neighbors(adj, current_index, 1)
            Local_1hop = get_gradient_two_layer(features[neighbor1], adj[current_index][:,neighbor1], adj[current_index][:,current_index], len(current_index))
 
            neighbor2 = get_K_hop_neighbors(adj, current_index, 2)

            Local_2hop = get_gradient_two_layer(features[neighbor2], adj[neighbor1][:,neighbor2], adj[current_index][:,neighbor1], len(current_index))
            #Local_comm = get_gradient_two_layer(features[neighbor2], adj[neighbor2][:,neighbor2], adj[neighbor2][:,neighbor2], len(neighbor2))

            Global = get_gradient_two_layer(features, adj, adj, len(adj))

            #print("Local_0hop", float(torch.norm(Local_0hop)))
            #print("Local_1hop", float(torch.norm(Local_1hop)))
            #print("Local_2hop", float(torch.norm(Local_2hop)))
            #print("Global", float(torch.norm(Global)))

            #print("Norm Local_comm Local_no_comm", float(torch.norm(Local_comm - Local_no_comm)))
            
            
            Norm_Local_0hop_Global.append(float(torch.norm(Local_0hop - Global)))
            Norm_Local_1hop_Global.append(float(torch.norm(Local_1hop - Global)))
            Norm_Local_2hop_Global.append(float(torch.norm(Local_2hop - Global)))
    print("Norm Local_0hop Global", np.array(Norm_Local_0hop_Global).sum() / len(Norm_Local_0hop_Global))
    print("Norm Local_1hop Global", np.array(Norm_Local_1hop_Global).sum() / len(Norm_Local_1hop_Global))
    print("Norm Local_2hop Global", np.array(Norm_Local_2hop_Global).sum() / len(Norm_Local_2hop_Global))
    Y_Norm_Local_0hop_Global.append(np.array(Norm_Local_0hop_Global).sum() / len(Norm_Local_0hop_Global))
    Y_Norm_Local_1hop_Global.append(np.array(Norm_Local_1hop_Global).sum() / len(Norm_Local_1hop_Global))
    Y_Norm_Local_2hop_Global.append(np.array(Norm_Local_2hop_Global).sum() / len(Norm_Local_2hop_Global))
            
            
np.save(dataset_name + "_Norm Local_0hop Global.npy", Y_Norm_Local_0hop_Global)
np.save(dataset_name + "_Norm Local_1hop Global.npy", Y_Norm_Local_1hop_Global)
np.save(dataset_name + "_Norm Local_2hop Global.npy", Y_Norm_Local_2hop_Global)


import matplotlib.pyplot as plt
X = np.array(range(0,11,1)) / 10

plt.plot(X, Y_Norm_Local_0hop_Global)
plt.plot(X, Y_Norm_Local_1hop_Global)
plt.plot(X, Y_Norm_Local_2hop_Global)

In [ ]:
plt.style.use('seaborn-whitegrid')
dataset_name = 'cora'
Y_Norm_Local_0hop_Global = np.load(dataset_name + "_Norm Local_0hop Global.npy")
Y_Norm_Local_1hop_Global = np.load(dataset_name + "_Norm Local_1hop Global.npy")
Y_Norm_Local_2hop_Global = np.load(dataset_name + "_Norm Local_2hop Global.npy")

import matplotlib.pyplot as plt
X = np.array(range(0,11,1)) / 10

plt.plot(X, Y_Norm_Local_0hop_Global, 's-', label = 'FedGCN(0-hop)', markersize=8)
plt.plot(X, Y_Norm_Local_1hop_Global, '+-', label = 'FedGCN(1-hop)', markersize=8)
plt.plot(X, Y_Norm_Local_2hop_Global, '*-', label = 'FedGCN(2-hop)', markersize=8)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)

plt.legend(fontsize=20, frameon=True, loc = 'best')
from matplotlib import rcParams
rcParams.update({'figure.autolayout': True})



plt.xlabel('IID Degree', fontsize=25)
plt.ylabel(r'$\mathit{\|\|\nabla {F_k}(\mathbf{w_k}) - \nabla {f}(\mathbf{w})\|\|}$', fontsize=25)
plt.legend(fontsize=20, frameon=True, loc = 'best')
from matplotlib import rcParams
rcParams.update({'figure.autolayout': True})
plt.savefig("cora_norm_data_distribution.png", format='png')

In [ ]:
np.random.seed(42)
torch.manual_seed(42)
dataset_name='citeseer'
args_normalize = True
Y_Norm_Local_0hop_Global = []
Y_Norm_Local_1hop_Global = []
Y_Norm_Local_2hop_Global = []

for percent in range(0, 11, 1):
    
    iid_percent = percent / 10
    K_over_class_num = 1
    features, adj, labels, one_hot_labels, split_data_indexes, K = get_graph(dataset_name, iid_percent, K_over_class_num)
    if dataset_name=='simulate':
            feature = one_hot_labels
    if args_normalize:
        adj = normalize(adj)
    Norm_Local_0hop_Global = []
    Norm_Local_1hop_Global = []
    Norm_Local_2hop_Global = []
    
    

    for i in range(len(split_data_indexes)):
            current_index = split_data_indexes[i]
            Local_0hop = get_gradient_two_layer(features[current_index], adj[current_index][:,current_index], adj[current_index][:,current_index], len(current_index))

            neighbor1 = get_K_hop_neighbors(adj, current_index, 1)
            Local_1hop = get_gradient_two_layer(features[neighbor1], adj[current_index][:,neighbor1], adj[current_index][:,current_index], len(current_index))
 
            neighbor2 = get_K_hop_neighbors(adj, current_index, 2)

            Local_2hop = get_gradient_two_layer(features[neighbor2], adj[neighbor1][:,neighbor2], adj[current_index][:,neighbor1], len(current_index))
            #Local_comm = get_gradient_two_layer(features[neighbor2], adj[neighbor2][:,neighbor2], adj[neighbor2][:,neighbor2], len(neighbor2))

            Global = get_gradient_two_layer(features, adj, adj, len(adj))

            #print("Local_0hop", float(torch.norm(Local_0hop)))
            #print("Local_1hop", float(torch.norm(Local_1hop)))
            #print("Local_2hop", float(torch.norm(Local_2hop)))
            #print("Global", float(torch.norm(Global)))

            #print("Norm Local_comm Local_no_comm", float(torch.norm(Local_comm - Local_no_comm)))
            
            
            Norm_Local_0hop_Global.append(float(torch.norm(Local_0hop - Global)))
            Norm_Local_1hop_Global.append(float(torch.norm(Local_1hop - Global)))
            Norm_Local_2hop_Global.append(float(torch.norm(Local_2hop - Global)))
    print("Norm Local_0hop Global", np.array(Norm_Local_0hop_Global).sum() / len(Norm_Local_0hop_Global))
    print("Norm Local_1hop Global", np.array(Norm_Local_1hop_Global).sum() / len(Norm_Local_1hop_Global))
    print("Norm Local_2hop Global", np.array(Norm_Local_2hop_Global).sum() / len(Norm_Local_2hop_Global))
    Y_Norm_Local_0hop_Global.append(np.array(Norm_Local_0hop_Global).sum() / len(Norm_Local_0hop_Global))
    Y_Norm_Local_1hop_Global.append(np.array(Norm_Local_1hop_Global).sum() / len(Norm_Local_1hop_Global))
    Y_Norm_Local_2hop_Global.append(np.array(Norm_Local_2hop_Global).sum() / len(Norm_Local_2hop_Global))
            
            
np.save(dataset_name + "_Norm Local_0hop Global.npy", Y_Norm_Local_0hop_Global)
np.save(dataset_name + "_Norm Local_1hop Global.npy", Y_Norm_Local_1hop_Global)
np.save(dataset_name + "_Norm Local_2hop Global.npy", Y_Norm_Local_2hop_Global)


import matplotlib.pyplot as plt
X = np.array(range(0,11,1)) / 10

plt.plot(X, Y_Norm_Local_0hop_Global)
plt.plot(X, Y_Norm_Local_1hop_Global)
plt.plot(X, Y_Norm_Local_2hop_Global)

In [ ]:
np.random.seed(42)
torch.manual_seed(42)
dataset_name='pubmed'
args_normalize = True
Y_Norm_Local_0hop_Global = []
Y_Norm_Local_1hop_Global = []
Y_Norm_Local_2hop_Global = []

for percent in range(0, 11, 1):
    
    iid_percent = percent / 10
    K_over_class_num = 1
    features, adj, labels, one_hot_labels, split_data_indexes, K = get_graph(dataset_name, iid_percent, K_over_class_num)
    if dataset_name=='simulate':
            feature = one_hot_labels
    if args_normalize:
        adj = normalize(adj)
    Norm_Local_0hop_Global = []
    Norm_Local_1hop_Global = []
    Norm_Local_2hop_Global = []
    
    

    for i in range(len(split_data_indexes)):
            current_index = split_data_indexes[i]
            Local_0hop = get_gradient_two_layer(features[current_index], adj[current_index][:,current_index], adj[current_index][:,current_index], len(current_index))

            neighbor1 = get_K_hop_neighbors(adj, current_index, 1)
            Local_1hop = get_gradient_two_layer(features[neighbor1], adj[current_index][:,neighbor1], adj[current_index][:,current_index], len(current_index))
 
            neighbor2 = get_K_hop_neighbors(adj, current_index, 2)

            Local_2hop = get_gradient_two_layer(features[neighbor2], adj[neighbor1][:,neighbor2], adj[current_index][:,neighbor1], len(current_index))
            #Local_comm = get_gradient_two_layer(features[neighbor2], adj[neighbor2][:,neighbor2], adj[neighbor2][:,neighbor2], len(neighbor2))

            Global = get_gradient_two_layer(features, adj, adj, len(adj))

            #print("Local_0hop", float(torch.norm(Local_0hop)))
            #print("Local_1hop", float(torch.norm(Local_1hop)))
            #print("Local_2hop", float(torch.norm(Local_2hop)))
            #print("Global", float(torch.norm(Global)))

            #print("Norm Local_comm Local_no_comm", float(torch.norm(Local_comm - Local_no_comm)))
            
            
            Norm_Local_0hop_Global.append(float(torch.norm(Local_0hop - Global)))
            Norm_Local_1hop_Global.append(float(torch.norm(Local_1hop - Global)))
            Norm_Local_2hop_Global.append(float(torch.norm(Local_2hop - Global)))
    print("Norm Local_0hop Global", np.array(Norm_Local_0hop_Global).sum() / len(Norm_Local_0hop_Global))
    print("Norm Local_1hop Global", np.array(Norm_Local_1hop_Global).sum() / len(Norm_Local_1hop_Global))
    print("Norm Local_2hop Global", np.array(Norm_Local_2hop_Global).sum() / len(Norm_Local_2hop_Global))
    Y_Norm_Local_0hop_Global.append(np.array(Norm_Local_0hop_Global).sum() / len(Norm_Local_0hop_Global))
    Y_Norm_Local_1hop_Global.append(np.array(Norm_Local_1hop_Global).sum() / len(Norm_Local_1hop_Global))
    Y_Norm_Local_2hop_Global.append(np.array(Norm_Local_2hop_Global).sum() / len(Norm_Local_2hop_Global))
            
            
np.save(dataset_name + "_Norm Local_0hop Global.npy", Y_Norm_Local_0hop_Global)
np.save(dataset_name + "_Norm Local_1hop Global.npy", Y_Norm_Local_1hop_Global)
np.save(dataset_name + "_Norm Local_2hop Global.npy", Y_Norm_Local_2hop_Global)


import matplotlib.pyplot as plt
X = np.array(range(0,11,1)) / 10

plt.plot(X, Y_Norm_Local_0hop_Global)
plt.plot(X, Y_Norm_Local_1hop_Global)
plt.plot(X, Y_Norm_Local_2hop_Global)